In [2]:
import torch
import os
from torch import linalg as LA
import numpy as np
print(torch.cuda.is_available())

True


/home/ziruiqiu/anaconda3/envs/DL2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device='cuda' 

# Take ρ(x) = tanh(x). Consider the 1-hidden layer neural network
# ŷ i = w 2 T ρ(W 1 x i + b 1 ) + b 2
# W is 20 × 10 and w 2 is a vector of size 20. x is a vector of size 10. b 1 and b 2 are vectors of size 20 and 1 respectively.

# Define the parameters
param_dict = {
    'W1': torch.randn(20, 10, device=device, requires_grad=True),
    'b1': torch.randn(10, device=device, requires_grad=True),
    'W2': torch.randn(20, 20, device=device, requires_grad=True),
    'b2': torch.randn(20, device=device, requires_grad=True)
}   


# Define the network without using any PyTorch modules
def my_nn(input, param_dict):

    # Reshape the input image from HxW to a flat vector of size H*W
    x = input.view(-1, 20)
    # print(x.shape)
    # print(param_dict['W1'].shape)
    # print(param_dict['b1'].shape)
    tanh = torch.nn.Tanh()

    x = torch.matmul(x, param_dict['W1']) + param_dict['b1']

    x = tanh(x)
    x = x.view(-1, 20)
    # print(x.shape)
    # print(param_dict['W2'].shape)
    # print(param_dict['b2'].shape)
    
    x = torch.matmul(x, param_dict['W2']) + param_dict['b2']
    x = x.view(-1, 100)
    #print(x.shape)
    return x

# The
# absolute loss given
# l(ŷ, y) = |ŷ − y|
# Consider the cost function J = 1/N ∑ i=1 l(ŷ , y )
# Derive an expression for
# ∂J/∂W1,∂J/∂W2,∂J/∂b1,∂J/∂b2
# and implement it in PyTorch.


# Define the loss function
def my_loss(y_hat, y):
    return torch.mean(torch.abs(y_hat - y))

# Define the gradient computation by hand
def compute_gradients( param_dict, x, y):
    W1, b1, W2, b2 = param_dict.values()
    print(x.shape)
    print(y.shape)
    print(W1.shape)
    print(b1.shape)
    print(W2.shape)
    print(b2.shape)
    
    N = x.shape[0]
    x = input.view(-1, 20)
    z1 = torch.matmul(x ,W1) + b1
    a1 = torch.tanh(z1)
    a1 = a1.view(-1, 20)
    z2 = torch.matmul(a1, W2) + b2
    y_pred = z2.view(-1, 100)
    
    print(y_pred.shape)
    sign = torch.sign(y_pred - y)
    dz2 = sign / N
    a1 = a1.view(-1, 10)
    dW2 = torch.matmul(a1, dz2)
    db2 = dz2.sum(dim=1)
    W2 = W2.view(-1, 10)
    da1 = torch.matmul(W2, dz2)

    a1 = a1.view(-1, 10)
    print(da1.shape)
    print((1 - a1**2).shape)
    dz1 = da1 * (1 - a1**2)
    dW1 = torch.matmul(dz1, x)
    db1 = dz1.sum(dim=1)

    return np.array([dW1, dW2, db1, db2])

# Define the training loop
def train_loop(param_dict, input, target, num_epochs=100, learning_rate=1e-3):
    for epoch in range(num_epochs):
        # gradient calulated by hand
        # hand_grad = compute_gradients(param_dict, input, target)
        # print(hand_grad)

        # gradient calculated by pytorch
        y_hat = my_nn(input, param_dict)
        loss = my_loss(y_hat, target)
        loss.backward()

        auto_W1_norm = LA.norm(param_dict['W1'].grad)
        auto_W2_norm = LA.norm(param_dict['W2'].grad)
        auto_b1_norm = LA.norm(param_dict['b1'].grad)
        auto_b2_norm = LA.norm(param_dict['b2'].grad)
        
        with torch.no_grad():
            for param in param_dict.values():
                param -= learning_rate * param.grad
                param.grad.zero_()
        if epoch % 10 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
    print(auto_W1_norm)
    print(auto_W2_norm)
    print(auto_b1_norm)
    print(auto_b2_norm)
    return param_dict

# Generate some random data
input = torch.randn(100, 20, device=device)
target = torch.randn(100, device=device)

# Train the network
train = train_loop(param_dict, input, target)


Epoch 0, Loss 3.398737
Epoch 10, Loss 3.397670
Epoch 20, Loss 3.396602
Epoch 30, Loss 3.395534
Epoch 40, Loss 3.394464
Epoch 50, Loss 3.393394
Epoch 60, Loss 3.392324
Epoch 70, Loss 3.391253
Epoch 80, Loss 3.390182
Epoch 90, Loss 3.389112
tensor(0.2334, device='cuda:0')
tensor(0.2178, device='cuda:0')
tensor(0.0583, device='cuda:0')
tensor(0.0430, device='cuda:0')


In [4]:
# Train this model on the sklearn California Housing Prices datasets.
# • For this you may use the optimizer and learning rates of your choice and train
# for 20-50 epochs.
# • Take half the data for training and half for testing.
# • Create a validation set from the training set and use it to select a good learning
# rate.
# • You might want to use the convenient Xavier initialization.
# • You are free to use the torch.optim package for this part.
# • To speed up things, run the training loop by batches (e.g. 4, 8, 32, 64, etc.).
# PyTorch’s DataLoader would be a useful tool to easily fetch a predefined set
# of batches per training iteration.
# • Report the mean squared error on the train and test set after each epoch.
# • You will need to adjust the size of W 1 to fit the size of this data.

# Load the data
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np

data = fetch_california_housing()
X = data['data']
y = data['target']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Normalize the data
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test = (X_test - np.mean(X_test, axis=0)) / np.std(X_test, axis=0)

# Convert to torch tensors
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)

# Define the network
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.linear1 = torch.nn.Linear(8, 20)
        self.linear2 = torch.nn.Linear(20, 1)
        self.tanh = torch.nn.Tanh()
    def forward(self, x):
        x = self.linear1(x)
        x = self.tanh(x)
        x = self.linear2(x)
        return x
    
# Define the loss function
def my_loss(y_hat, y):
    return torch.mean(torch.abs(y_hat - y))

# Define the training loop
def train_loop(model, input, target, num_epochs=1000, learning_rate=1e-3):
    for epoch in range(num_epochs):
        y_hat = model(input)
        loss = my_loss(y_hat, target)
        loss.backward()
        with torch.no_grad():
            for param in model.parameters():
                param -= learning_rate * param.grad
                param.grad.zero_()
        if epoch % 100 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
    return model

# Train the network
model = MyNet().to(device)
model = train_loop(model, X_train, y_train)

# Evaluate the network
y_hat = model(X_test)
loss = my_loss(y_hat, y_test)
print('Test loss: %f' % loss)



Epoch 0, Loss 2.186485
Epoch 100, Loss 2.001607
Epoch 200, Loss 1.813745
Epoch 300, Loss 1.624517
Epoch 400, Loss 1.448838
Epoch 500, Loss 1.301474
Epoch 600, Loss 1.186870
Epoch 700, Loss 1.100833
Epoch 800, Loss 1.037673
Epoch 900, Loss 0.992444
Test loss: 0.962929


In [60]:
# question 2
# Consider the neural network
# f(x) = WF ρ ◦ WL . . . ρ ◦ Wi . . . ρ ◦ W2ρ ◦ W1x
# where W1 is K × D, Wi
# is K × K for i > 1, and WF is P × K. Note f : RD → RP .
# Take ρ(x) = tanh(x). We will examine different ways to compute the Jacobian ∂f(x)/∂x .
# Use torch tensors to write a function which computes the Jacobian,
# ∂f(x)/∂x , using backward mode automatic differentiation for a given value of x and
# W1, . . . , Wi, . . . , WF where the given matrices are specified by a dictionary of torch
# tensors. Implement and test this for L = 3. Your function should only make use
# of basic matrix operations (e.g. torch.matmul(), torch.tanh(), etc). You may not
# use autograd or autograd.jacobian for your implementation (but you can use them
# to unit test your answer). Test it for the case of D = 2, K = 30, P = 10, your
# solution does not have to cover all edge cases of K, P,D it is sufficient it works on
# the ones provided here.

# Define the network
def my_nn(x, param_dict):
    for i in range(1, L):
        x = torch.matmul(param_dict['W' + str(i)], x)
        x = torch.tanh(x)
    x = torch.matmul(param_dict['WF'], x)
    return x

# Define the Jacobian function
def my_jacobian(x, param_dict):
    y = my_nn(x, param_dict)
    J = torch.zeros(y.shape[0], x.shape[0])
    for i in range(y.shape[0]):
        y[i].backward(retain_graph=True)
        J[i, :] = x.grad
        x.grad.zero_()
    return J

# Define the parameters
D = 2
K = 30
P = 10
L = 3
param_dict = {}
param_dict['W1'] = torch.randn(K, D, device=device, requires_grad=True)
for i in range(2, L):
    param_dict['W' + str(i)] = torch.randn(K, K, device=device, requires_grad=True)
param_dict['WF'] = torch.randn(P, K, device=device, requires_grad=True)

# Define the input
x = torch.randn(D, device=device, requires_grad=True)

# Compute the Jacobian
J = my_jacobian(x, param_dict)
print(J)







torch.Size([2])
torch.Size([30, 2])
torch.Size([30])
torch.Size([30, 30])
tensor([[ -1.8950,  -2.7902],
        [ -9.1231,  -6.4170],
        [-17.9673, -11.5019],
        [  3.8187,  -0.6564],
        [ -0.0287,  -5.0156],
        [ -9.5783,  -7.7911],
        [ -6.1696,  -3.3152],
        [-12.8570,  -5.3742],
        [ -2.7195,   4.2060],
        [-14.1800,  -7.3539]])


In [62]:
# (b) (12 points) Implement a function using torch tensors and forward mode automatic
# differentiation to compute ∂f(x)/ ∂x . Validate (with assert statements) for several test
# cases that your answer matches the function (b) for L = 3. Hint: You must
# calculate the derivatives and the network’s output in the same forward pass (unlike
# backward differentiation where you need two loops, one for the forward pass and
# one for calculating the gradient)

# Define the Jacobian function
def my_jacobian(x, param_dict):
    y = my_nn(x, param_dict)
    J = torch.zeros(y.shape[0], x.shape[0])
    for i in range(y.shape[0]):
        y[i].backward(retain_graph=True)
        J[i, :] = x.grad
        x.grad.zero_()
    return J

# Define the parameters
D = 2
K = 30
P = 10
L = 3
param_dict = {}
param_dict['W1'] = torch.randn(K, D, device=device, requires_grad=True)
for i in range(2, L):
    param_dict['W' + str(i)] = torch.randn(K, K, device=device, requires_grad=True)
param_dict['WF'] = torch.randn(P, K, device=device, requires_grad=True)

# Define the input
x = torch.randn(D, device=device, requires_grad=True)

# Compute the Jacobian
J = my_jacobian(x, param_dict)
print(J)




torch.Size([2])
torch.Size([30, 2])
torch.Size([30])
torch.Size([30, 30])
tensor([[-3.6832,  3.1388],
        [ 2.1358, -3.0074],
        [-5.7035,  3.5083],
        [ 4.5243, -3.9619],
        [ 3.4845, -4.2608],
        [-1.6215,  1.7399],
        [ 2.4567, -0.5928],
        [ 5.4941, -5.6483],
        [ 0.4993,  1.7094],
        [-5.2610,  3.5687]])
